In [78]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,root_mean_squared_error
import plotly.express as px

In [79]:
df = pd.read_csv("Data4.csv")
df = df.drop("Sl.", axis=1)
# df = df.drop(1, axis=0, inplace=True)   ## For Droping a row with index 1.
df.head()

,x1,x2,x3,y
0,0.1,0.311089,0.390541,14.705876
1,0.2,0.639066,1.389918,26.585100
2,0.3,0.840228,1.939903,36.791509
3,0.4,1.299710,2.153009,34.090806
4,0.5,0.941784,0.945136,31.118089


In [80]:
X0 = [float(1) for i in df["x1"]]
X1 = [float(i) for i in df["x1"]]
X2 = [float(i) for i in df["x2"]]
X3 = [float(i) for i in df["x3"]]
Y = [float(i) for i in df["y"]]

X_list = [X0, X1, X2, X3] 
# len(df.columns)
# len(df)

**This is implimentation of simple linear regression using matrix inverse; For multiple input and one output.**

In [81]:
n = len(df) #100
k = len(df.columns) #4
Vector_Y = []

for i in range(k):

    var = []
    for j in range(n):
        p = X_list[i][j] * Y[j]
        var.append(p)

    Vector_Y.append(sum(var))

print(Vector_Y)


[10441.800450159999, 66074.55450871799, 71379.2727789428, 76343.57296321742]


In [82]:
Xi = [] # for eg. X1
for i in range(1, k):
    Xi.append(sum(X_list[i]))


XiXi = [] # for eg. X1X1
for i in range(1, k):
    var = []
    for j in range(n):
        p = X_list[i][j] * X_list[i][j]
        var.append(p)
    XiXi.append(sum(var))


odds = []
l = [[1,2], [2,3], [3,1]]

for i in range(len(l)):
    f =l[i][0]
    s =l[i][1]

    ll = []
    for j in range(n):
        p = X_list[f][j] * X_list[s][j]
        ll.append(p)

    odds.append(sum(ll))

print("Xi :", Xi)
print("Xixi :", XiXi)
print("odds :", odds)

matrix_content = [100.0, 505.0, 554.0457727056, 606.1264987875, XiXi[0], odds[0], odds[2], XiXi[1], odds[1], XiXi[2]]

Xi : [505.0, 554.0457727055998, 606.1264987875]
Xixi : [3383.5000000000005, 3929.682783920666, 4472.487676145371]
odds : [3641.0164390424306, 4171.374220465155, 3863.4271194983494]


In [83]:
Matrix_X = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
]

# Fill the upper and lower triangular parts symmetrically
index = 0
for i in range(k):
    for j in range(i, k):
        Matrix_X[i][j] = matrix_content[index]
        Matrix_X[j][i] = matrix_content[index]
        index += 1
for row in Matrix_X:
    print(row)

[100.0, 505.0, 554.0457727056, 606.1264987875]
[505.0, 3383.5000000000005, 3641.0164390424306, 3863.4271194983494]
[554.0457727056, 3641.0164390424306, 3929.682783920666, 4171.374220465155]
[606.1264987875, 3863.4271194983494, 4171.374220465155, 4472.487676145371]


In [84]:
# A_Transpose = np.transpose(A) or A.T #Here A should be np.array([[], [], []])
# New_X is a symmetrix matrix so It's transpose should be same as X.
Matrix_X_Transpose = Matrix_X

# Matrix multiplication :
# numpy.matmul(A, B) ; Here A, B need not be np.array, Simple 2-D list.
# A @ B @ C  ; Here A, B need to be np.array and First it will multiply (AXB) then (result X C).
Matrix_A = np.matmul(Matrix_X_Transpose, Matrix_X)

# Inverse of matrix A where A should not be singular.
Matrix_A_inverse = np.linalg.inv(Matrix_A)

result = np.matmul(np.matmul(Matrix_A_inverse, Matrix_X_Transpose), Vector_Y)

print("value of beta0 :", result[0])
print("value of beta1 :", result[1])
print("value of beta2 :", result[2])
print("value of beta3 :", result[3])

value of beta0 : 13.239477941118366
value of beta1 : 6.132437615313393
value of beta2 : 2.392265549258809
value of beta3 : 7.746810380660236


In [85]:
predicted_Y = []
for i in range(100):
  r1 = result[0] + result[1] * X1[j] + result[2] * X2[j] + result[3] * X3[j]
  predicted_Y.append(r1)

# Verification

In [86]:
df.head()

,x1,x2,x3,y
0,0.1,0.311089,0.390541,14.705876
1,0.2,0.639066,1.389918,26.585100
2,0.3,0.840228,1.939903,36.791509
3,0.4,1.299710,2.153009,34.090806
4,0.5,0.941784,0.945136,31.118089


In [87]:
X = df[['x1', 'x2', 'x3']]
Y = df['y']

In [88]:
regr = LinearRegression()

regr.fit(X, Y)
print("Slopes: ",regr.coef_)
print("Intercept: ",regr.intercept_)

Slopes:  [6.13243763 2.39226554 7.74681038]
Intercept:  13.239477824445359


In [89]:
predicted_Y_SK = []
for i in range(100):
  r1 = 13.239477824445359 + 6.13243763 * X1[j] + 2.39226554 * X2[j] + 7.74681038 * X3[j]
  predicted_Y_SK.append(r1)

In [98]:
print(predicted_Y)
print(predicted_Y_SK)

[35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087438333, 35.48066087

In [99]:
# Error between "prediction by SKLearn Regression model" and "given data."

mae = mean_absolute_error(y_true=Y,y_pred=predicted_Y_SK)
mse = mean_squared_error(y_true=Y,y_pred=predicted_Y_SK)
rmse = root_mean_squared_error(y_true=Y,y_pred=predicted_Y_SK)

print("MAE:",mae)
print("MSE:",mse)
print("RMSE:",rmse)


MAE: 69.95183677668845
MSE: 6940.052463331254
RMSE: 83.30697727880452


In [100]:
# Error between "prediction by My Own Regression model" and "given data."

mae = mean_absolute_error(y_true=Y,y_pred=predicted_Y)
mse = mean_squared_error(y_true=Y,y_pred=predicted_Y)
rmse = root_mean_squared_error(y_true=Y,y_pred=predicted_Y)

print("MAE:",mae)
print("MSE:",mse)
print("RMSE:",rmse)

MAE: 69.95183666983033
MSE: 6940.052446199825
RMSE: 83.30697717598343
